# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


In [4]:
# test
# my explore data, understanding the edata

#type(data) #helpers.Dataset
#data[0]

# 'b100-36703':
#             Sentence(
#                 words=('As', 'Pip', 'agonizes', 'over', 'the', (long list) 
#                 tags=('ADP', 'NOUN', 'VERB', 'ADP', 'DET', (long list)
#             )
# 'b100-28124': 
#             Sentence(
#                 words=



### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. Review the reference below, then run and review the next few cells to make sure you understand the interface before moving on to the next step.

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [5]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [6]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [7]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [8]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

## Step 2: Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [9]:
# test 2d dict - to delete
aaa = {}
if 'a' not in aaa:
    print ("aaa['a'] was None")
    aaa['a'] = {}
aaa['a']['b'] = 1
print (aaa['a']['b'])

aaa['a'] was None
1


In [10]:
# check amount of data the code need to handle - to delete
# len(data.Y)
len(data.X)

57340

In [171]:
# dont run
return

# Data explore, Code try and testing
# to understand how it all works, before implement project requirememnt - def pair_counts()
# basically all the test code of pair_counts()
def my_data_analysis(sequences_A, sequences_B):
 
    # view data
    # print ('sequences_A 2d=', sequences_A)
    # print ('sequences_B 2d=', sequences_B)
    #print (type(sequences_B)) # <class 'tuple'>
    #print (type(sequences_B[0])) # <class 'tuple'>
    
    # flatten tuples in tuple to one dimension
    words = [w for s in sequences_B for w in s]
    tags = [t for s in sequences_A for t in s]
    # print ('words 1d=', words)
    # print ('tags 1d=', tags)
    utags = set(tags) 
    print ('number of all tags =',len(tags))
    print ('number of unique tags =',len(utags))
    
    # we need words, not duplicates
    uwords = set(words)
    print ('number of all words =', len(words))
    print ('number of unique words =', len(uwords))
    
    # try get one [tag][word] working
    sentence = sequences_B[0]
    word = sentence[7] # word=and
    print ('test word used:', word)

    # find all index of the same word
    word_indices = [i for i, w in enumerate(words) if w == word] 
    # print (len(word_indices)) # 1
    # print (type(word_indices)) # <class 'list'>
    # print ('word_indices=',word_indices)
    
    tag_of_word = tags[7]
    tag_word_indices = [i for i in word_indices if tags[i] == tag_of_word]
    # print ('tag_word_indices=',tag_word_indices)
    
    # check result is correct
    # for i in tag_word_indices:
    #     print ('word =', words[i], ', tag =', tags[i])
    # check 1d list, with all words
    # for i, w in enumerate(words):
    #     if w == word:
    #         print ('word =', words[i], ', tag =', tags[i])
    # print count length instead of return the list
    print ('[{0}][{1}]={2}'.format(tag_of_word, word, len(tag_word_indices)))
    
    ########################################
    # putting all test code above together
    ########################################
    # get all count of [tag][word] - project requirement
    # get list of unique words
    # get list of unique tags
    # init a new dictionary, to hold all [tag][word] with count
    # loop all words
    #   for each word, get all possible tags list
    #     for each possbile tag of current word
    #       count number of time [tag][word] occurs
    #       add this key value(count) to dictionary
    #       repeat on next word
    
    print ('#####################################')
    # flatten tuples in tuple to one dimension
    words = [w for s in sequences_B for w in s]
    tags = [t for s in sequences_A for t in s]
    
    # unique tags and unique words
    utags = set(tags) 
    uwords = set(words)

    tag_word_count = dict()
    #tag_word_count[tag][word] = count
    
    # loop all unique words
    for i, uw in enumerate(uwords):
        
#         print ('next unique word=', uw)
        # get all indices of the same unique word
        word_indices = [j for j, w in enumerate(words) if w == uw] 
#         print ('word_indices=', word_indices)
        
        # get possible all tags (duplicates exist) of each word, using each word occurances index
        for j in word_indices:
            all_tags_of_current_word = [t for k, t in enumerate(tags) if k == j] 
        
#         print ('all_tags_of_current_word=', all_tags_of_current_word)
        # for each tag, combine with word, count all existence
        # loop by each unique tag, and count number of occurance
        utags = set(all_tags_of_current_word)   
#         print ('utags of this word=', utags)
        for ut in utags:
            utag_count_of_current_word = all_tags_of_current_word.count(ut)
            # create new dict entry for each new tag
            if ut not in tag_word_count:
                tag_word_count[ut] = {}
            tag_word_count[ut][uw] = utag_count_of_current_word
    
    return tag_word_count

# new version to make code faster
def my_data_analysis_2(sequences_A, sequences_B):
 
    # flatten tuples in tuple to one dimension
    words = [w for s in sequences_B for w in s]
    tags = [t for s in sequences_A for t in s]
    
    # join the two lists side by side - [tag][word]
    # list() wrap zip() to access array like methods, as zip() object function is limited
    pairs = list(zip(tags, words))
#     print (len(pairs))
    
    # unique pairs, for looping, as we dont need to loop all pairs
    upairs = set(pairs)
#     print (len(upairs))
    
    # create 2d dictionary [tag][word]=count
    tag_word_count = dict()
    for up in upairs:
        index1 = up[0]
        index2 = up[1]
#         print (index1)
#         print (index2)
        # create index is needed for 2nd indices
        if index1 not in tag_word_count:
            tag_word_count[index1] = {}
        tag_word_count[index1][index2] = pairs.count(up)
    
    return tag_word_count
        
# emission_counts = my_data_analysis(data.training_set.Y[0:100], data.training_set.X[0:100])
# emission_counts = my_data_analysis(data.training_set.Y[0:5000], data.training_set.X[0:5000])
subset_tags = data.training_set.Y[:1000]
subset_words = data.training_set.X[:1000]
emission_counts = my_data_analysis_2(subset_tags, subset_words)

print (len(emission_counts))
print (max(emission_counts["NOUN"], key=emission_counts["NOUN"].get))
# 12
# time


12
time


In [90]:
# dont run
return

# this version takes too long, rewrite needed
# the result is only partially correct anyway
def pair_counts_v1(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # TODO: Finish this function!
 
    ########################################
    # get all count of [tag][word] - project requirement
    # get list of unique words
    # get list of unique tags
    # init a new dictionary, to hold all [tag][word] with count
    # loop all words
    #   for each word, get all possible tags list
    #     for each possbile tag of current word
    #       count number of time [tag][word] occurs
    #       add this key value(count) to dictionary
    #       repeat on next word
    ########################################
    
    # flatten tuples in tuple to one dimension
    words = [w for s in sequences_B for w in s]
    tags = [t for s in sequences_A for t in s]
    
    # unique tags and unique words
    utags = set(tags) 
    uwords = set(words)

    tag_word_count = dict()
    #tag_word_count[tag][word] = count
    
    # loop all unique words
    for i, uw in enumerate(uwords):
        
#         print ('next unique word=', uw)
        # get all indices of the same unique word
        word_indices = [j for j, w in enumerate(words) if w == uw] 
#         print ('word_indices=', word_indices)
        
        # get possible all tags (duplicates exist) of each word, using each word occurances index
        for j in word_indices:
            all_tags_of_current_word = [t for k, t in enumerate(tags) if k == j] 
        
#         print ('all_tags_of_current_word=', all_tags_of_current_word)
        # for each tag, combine with word, count all existence
        # loop by each unique tag, and count number of occurance
        utags = set(all_tags_of_current_word)   
#         print ('utags of this word=', utags)
        for ut in utags:
            utag_count_of_current_word = all_tags_of_current_word.count(ut)
            # create new dict entry for each new tag
            if ut not in tag_word_count:
                tag_word_count[ut] = {}
            tag_word_count[ut][uw] = utag_count_of_current_word
    
    return tag_word_count
        
    
#     raise NotImplementedError


# Calculate C(t_i, w_i)
subset_tags = data.training_set.Y[:1000]
subset_words = data.training_set.X[:1000]
emission_counts = pair_counts_v1(subset_tags, subset_words)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

AssertionError: Hmmm...'time' is expected to be the most common NOUN.

In [11]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # TODO: Finish this function!
 
    # flatten tuples in tuple to one dimension list
    words = [w for s in sequences_B for w in s]
    tags = [t for s in sequences_A for t in s]
    
    # zip() - join the two lists side by side - [tag][word]
    # list() wrap zip() to access array like methods, as zip() object functions is limited
    pairs = list(zip(tags, words))
    
    # unique pairs, for dictionary unique keys
    upairs = set(pairs)
    
    # create 2d dictionary [tag][word]=count
    tag_word_count = dict()
    
    # loop to count and record occurance of each unique pairs of [tag][word]
    for up in upairs:
        tag = up[0] #index1
        word = up[1] #index2
        
#     for i, pair in enumerate(data.stream()):
#         tag = pair[1] #index1
#         word = pair[0] #index2

        # create new dict per index is needed, or exception will throw
        if tag not in tag_word_count:
            tag_word_count[tag] = {}

        # add count to dictionary
        tag_word_count[tag][word] = pairs.count(up)
    
    return tag_word_count
    
# Calculate C(t_i, w_i)
subset_tags = data.training_set.Y #[:1000]
subset_words = data.training_set.X #[:1000]
emission_counts = pair_counts(subset_tags, subset_words)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

In [ ]:
# run time is about 1-2 hours!

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [ ]:
# test code

# init a tuple of word with its most frequent tag
# WordTag = namedtuple('WordTag', 'word tag')
# aaa = WordTag(word='simspon', tag='NOUN')
    
# print (word_counts['the'])
       
# filter by each word
# aaa = [ v for k, v in word_counts.items() ] # if v == 'the' ] 
# print (aaa)

# try group by dict 2nd index word
# from itertools import groupby
# from operator import itemgetter

# grouper = itemgetter('word')

In [90]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

# subset_tags = data.training_set.Y #[:1000]
# subset_words = data.training_set.X #[:1000]
# word_counts = pair_counts(subset_tags, subset_words)

# reuse result from code above
word_counts = emission_counts

def word_most_freq_tag():

    word_mtag_name = dict() # [word]=most freq tag
    word_mtag_count = dict() # [word][most freq tag]=count, act as a temp placeholder for the most freq tag value 

    for tag, words in word_counts.items():
        for word, value in words.items():
    #         print ('{0},{1}={2}'.format(tag, word, value))
            if word not in word_mtag_name:
                word_mtag_count[word] = value
                word_mtag_name[word] = tag
            elif word_mtag_count[word] < value:
                word_mtag_count[word] = value
                word_mtag_name[word] = tag
#                 print ('new found most freq tag is {0} for word {1} with count {2}'.format(tag, word, value))
    
    return word_mtag_name

# def word_most_freq_tag_2():

#     word_maxtag = dict() # return dict of: [word]=tag(most freq)

#     # using
#     # word_counts = [tag][word] = count
        
#     for uword in data.vocab:
        
#         list(word_counts)[:][uword]
        
#         for word, value in words.items():

    
#     return word_mtag_name
    
mfc_table = word_most_freq_tag()

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

In [91]:
# code run above have correct result and passed assert test, but pair_counts() is slow, rewrite if time allow

In [92]:
# check result
# print (mfc_table)

In [93]:
# test - return max value index in dict
numbers = {'a':99, 'b':101, 'c': 100}
print (numbers)
print (max(numbers)) # return c index, as c is 'bigger' than both b and a
print (max(numbers.values())) # return 101 value

# try code but dont work
# print (numbers.index(max(numbers.values())))
# iii = [i for (i, n) in numbers.iteritems() if n == 100]

index_of_max_value = max(numbers, key=numbers.get)
print (index_of_max_value)

{'b': 101, 'c': 100, 'a': 99}
c
101
b


In [94]:
# for uword in data.vocab:
# #     print (uword)
#     word_tags = {k: v for k, v in word_counts.items() if v[]}
#     print (word_tags)

# for k, v in word_counts.items():
#     print (v)

    

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [95]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [96]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print()
    print("Actual words:\n--------------")
    print(data.sentences[key].words)
    print()
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144


Actual words:
--------------
('and', 'August', '15', ',', 'November', '15', ',', 'February', '17', ',', 'and', 'May', '15', ',', '(', 'Cranston', ')', '.')

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146


Actual words:
--------------
('She', 'had', 'the', 'opportunity', 'that', 'few', 'clever', 'women', 'can', 'resist', ',', 'of', 'showing', 'her', 'superiority', 'in', 'argument', 'over', 'a', 'man', '.')

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [97]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [98]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


## Step 3: Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmax}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [99]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO: Finish this function!
    
    # flatten 2d to 1d, for better access
    tags = [t for s in sequences for t in s]
    
    # list of unique tags for dict keys
    utags = set(tags)
        
    # tags occurance count dictionary
    unigram_counts_dict = dict()
    
    # count each unique tag in all tags list
    for ut in utags:
        unigram_counts_dict[ut] = tags.count(ut)
    
    return unigram_counts_dict
    

# TODO: call unigram_counts with a list of tag sequences from the training set
subset_tags = data.training_set.Y #[:1000]
tag_unigrams = unigram_counts(subset_tags)

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [100]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """

    # TODO: Finish this function!
    
    # The original pesudo code, later logic idea changed:
    # make a copy of the tags
    # shift copy index -1
    # create pair list, by zip both original and copied list
    # get unique pair list by set()
    # create dict for return result
    # loop unique pair list
    # count occurance of each unique pair in pair list
    
    ###
    # flatten 2d to 1d, for better access
    # tags = [t for s in sequences for t in s]

    # zip/pair with next index shift, (i, i+1), skip last index as no next index
    # start index pair = (None, tag) - not incuded
    # end index pair   = (tag, None) - skipped by last index - 1
    # pairs = [ (tags[i], tags[i+1]) for i, t in enumerate(tags) if i < (len(tags) - 1) ] 

    # print ('len(pairs)=', len(pairs)) #928457
    
    ###
    # fix potential problem from my own logic
    # 'sentence end tag' pairing with 'next sentence start tag'
    pairs = []
    for seq in sequences:
        next_seq_tag_pairs = [ (seq[i], seq[i+1]) for i, t in enumerate(seq) if i < (len(seq) - 1) ]
        pairs.extend(next_seq_tag_pairs)
        
    # print ('len(pairs)=', len(pairs)) #882586
    # yes it does reduce and correct tge incorrect tags
    # but this only increased the accuracy by just 0.01%, not enough, problem elsewhere
        
#         for i, tag in enumerate(seq):
#             if i < len(seq)-1):
#                 next_pair = seq[i], seq[i+1]
#                 pairs.append(next_pair)
    
    ###
    # check result data, first and last index
    #     print (pairs[0])
    #     print('-----')
    #     print (pairs[-1])
    #     print('-----')
    
    ###
    # list of unique pairs for dict keys
    upairs = set(pairs)
        
    # tags occurance count dictionary
    bigram_counts_dict = dict()
    
    # count each unique pairs in all pairs list
    for ut in upairs:
        bigram_counts_dict[ut] = pairs.count(ut)

    # check result data
    # print (bigram_counts_dict)
    # print('-----')

    return bigram_counts_dict
    

# TODO: call bigram_counts with a list of tag sequences from the training set
subset_tags = data.training_set.Y #[:1000]
tag_bigrams = bigram_counts(subset_tags)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

In [101]:
# check result in tag_bigrams
for i, v in enumerate(tag_bigrams):
    print (i, v, tag_bigrams[v])
    if i > 5:
       break


0 ('ADV', 'PRON') 2136
1 ('PRON', 'CONJ') 455
2 ('PRON', 'PRON') 318
3 ('ADP', 'DET') 52841
4 ('CONJ', 'PRON') 2058
5 ('ADV', 'CONJ') 789
6 ('VERB', '.') 11699


### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [102]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    # TODO: Finish this function!
        
    # get first word of each sequence
    start_tags = [s[0] for s in sequences]
        
    # list of unique start_tags for dict keys
    ustart_tags = set(start_tags)
        
    # start tags occurance count dictionary
    start_tags_counts_dict = dict()
    
    # count each unique start tag in all start_tags list
    for ust in ustart_tags:
        start_tags_counts_dict[ust] = start_tags.count(ust)

    # check result data
    # print (start_tags_counts_dict)
    # print('-----')

    return start_tags_counts_dict

# TODO: Calculate the count of each tag starting a sequence
subset_tags = data.training_set.Y #[:1000]
tag_starts = starting_counts(subset_tags)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

In [103]:
# test - explore result data in tag_starts

# view what tag in chosen index
print ('index 0 is', list(tag_starts)[0]) # 'ADP'
print ('index 1 is', list(tag_starts)[1]) # 'X'

# view count of some tags
print ('ADP count', tag_starts['ADP']) # 5583
print ('X count', tag_starts['X']) # 25

# all dict values
print ('count of each start tag = ', tag_starts.values())

# sum all values, need later in building hmm tagger
print ('count of all start tags = ', sum(tag_starts.values()))


index 0 is ADP
index 1 is PRON
ADP count 5583
X count 25
count of each start tag =  dict_values([5583, 7318, 2282, 1582, 9763, 25, 2080, 760, 4185, 1718, 4107, 6469])
count of all start tags =  45872


### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [104]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    # TODO: Finish this function!
    
    # the code is almost the same as starting_counts()
    # index used is now -1 instead of 0
    
    # get last word of each sequence
    end_tags = [s[-1] for s in sequences]
        
    # list of unique end_tags for dict keys
    uend_tags = set(end_tags)
        
    # end tags occurance count dictionary
    end_tags_counts_dict = dict()
    
    # count each unique end tag in all end_tags list
    for uet in uend_tags:
        end_tags_counts_dict[uet] = end_tags.count(uet)

    return end_tags_counts_dict

# TODO: Calculate the count of each tag ending a sequence
subset_tags = data.training_set.Y #[:1000]
tag_ends = ending_counts(subset_tags)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [108]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)

#####
# Analysis:
# ideas / logic for states:
#   loop unique tags
#   for each tag, calculate the probability of being each word, for all words
#   use warmup optional notebook code to get DiscreteDistribution and State objects
#   add to model states list

#####
# flat function
# def flat(sequences):
#     return [w for s in sequences for w in s]

#####
# variables
# utags = set(flat(data.training_set.Y)) # Y flatten
utags = data.tagset # provided property/function to achieve the same

# uwords = set(flat(data.training_set.X)) # X flatten
uwords = data.vocab # provided property/function to achieve the same

# total_no_of_tags = len(data.training_set.Y)
total_no_of_tags = data.N      # provided property/function to achieve the same

# quick tets smaller subset
# subset_tags = data.training_set.Y #[:200]
# subset_words = data.training_set.X #[:200]
# emission_counts = pair_counts(subset_tags, subset_words)

# emission_counts = word_counts

#####
# test returning the dictionary on the required format
# import numpy as np
# numbers = np.arange(10)
# result_list = {n: n**2 for n in numbers}
# print (result_list)
# return

#####
# Analysis of tag -> word probabilities
# each tag (state) have a list of probability of being a particular word, for all words
# tag is the state here
# words is the possible value of the state (tag)
def tag_is_words_probs(tag):
    # Analysis:
    # 1. example format on what to return: 
    #    tag = 'NOUN', return {"the": 0.012, "this": 0.009, "glass": 0.180}
    # 2. formula:
    #    P(w|t) = C(t, w) / C(t)
    #    C(t, w) by pair_counts(), use keys (tag, word) to get count value
    #    C(t) count of total number of tags
    # 3. pseudo code (kind of):
    # for tag 'NOUN'
    # list[w: prob]  = car: number of times the noun is car / number of tags, for all unique words  
    #                = car: emission_counts[NOUN][car] / number of tags, for all unique words

    # notice: we are using number of all tags, not unique tags. As probability depend on checking occurances of all data 
    
    # test simple / debug an issue
    # state_probs_list = {word: 1 for word in uwords} 

    # real deal
    # if statement at the end of [] checks if the word exist in tag-word pairs first, e.g. water will prob not ever be a verb
    # wrong denominator - total_no_of_tags
    # tag_is_words_probs_list = {word: (emission_counts[tag][word] / total_no_of_tags) for word in uwords if word in emission_counts[tag]}
    # use denominator - tag_unigrams[tag]
    tag_is_words_probs_list = {word: (emission_counts[tag][word] / tag_unigrams[tag]) for word in uwords if word in emission_counts[tag]}

    # check if all probability sum to 1 (100%), but is it 100% on the level above but no here? need more understanding!
    # loop dict, add up values
#     total_prob_value = 0
#     for k, v in state_probs_list.items():
#         total_prob_value += v
        
#     print ('total_prob_value=', total_prob_value)
    
    return tag_is_words_probs_list
    
######################## 
# emission probabilities
#   for each tag of all unique tag, get the tag is each words probabilities, for all words
# create state object, add state to model
# also store the states for later use in add transition (tag to tag) to model

tag_state_list = dict()

# for tag in list(utags)[:1]: # quick test
for tag in utags: # real deal
    # example what to return, from warm up optional notebook: {"yes": 0.1, "no": 0.9}
    emission_probabilities = tag_is_words_probs(tag)
  
    # print (emission_probabilities)

    tag_is_word_emissions = DiscreteDistribution(emission_probabilities)
    tag_is_word_state = State(tag_is_word_emissions, name=tag)

    basic_model.add_states(tag_is_word_state)
    
    tag_state_list[tag] = tag_is_word_state

####################################################################
# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
# (Hint: you may need to loop & add transitions

###########################
# next
# 1. none -> start_tag prob
# 2. tag -> end_tag prob
# 3. tag -> tag prob
# 4. transitions

# use lists created in previous code cells
#   tag_unigrams is single tag count list, e.g. list[tag] = 100
#   tag_bigrams is tag to tag count list, e.g. list[tag][tag] = 100
#   tag_starts is first tag count list, e.g. list[start_tag] = 100
#   tag_ends is end tag count list, e.g. list[end_tag] = 100

# observation probabilities, at the very first state - tag
# like the first day of the week monday, prob of being sunny
def tag_is_start_probs():
    # start tag is tag
    # P(t|start) = C(start, t) / C(start)
    
    # wrong denominator, should not be sum of all start tags
    # prob of each start tag = count each start tag occurance / sum of occurances of all start tag
    # count_start_tags = sum(tag_starts.values()) # sum of all occurances of start tags
    # state_is_start_probs_list = {start_tag: (tag_starts[start_tag] / count_start_tags) for start_tag in tag_starts}
    
    # denominator - should be occurance count of next unique start tag
    tag_is_start_probs_list = {tag: (tag_starts[tag] / tag_unigrams[tag]) for tag in tag_starts}
#     tag_is_start_probs_list = {start_tag: (tag_starts[start_tag] / len(data.training_set.Y) ) for start_tag in tag_starts}
    return tag_is_start_probs_list

def tag_is_end_probs():
    # end tag is tag
    # P(end|t) = C(t, end) / C(t)

    # code try but not following the formula provided
    # count_end_tags = sum(tag_ends.values()) # sum of all occurances of end tags
    # end_is_state_probs_list = {tag: (tag_ends[end_tag] / count_end_tags) for end_tag in tag_ends}
    
    # if the following works, then i dont really understand the maths, why logic is different to start tags? 
    # becos start tag have nothing before, so only uses the start tags as denominator?
    # and end tags have all possible unique tags before, so uses all uniques as denominator?
    
    # wrong denominator, should not be number of unique tags
    # count_utags = len(utags)
    # state_is_end_probs_list = {tag: (tag_ends[tag] / count_utags) for tag in utags}
    
    # denominator - should be occurance count of next unique tag
    # also, only need to loop the end tags, but not all unique tags
    tag_is_end_probs_list = {tag: (tag_ends[tag] / tag_unigrams[tag]) for tag in tag_ends}
    
    return tag_is_end_probs_list

def tag_to_tag_probs():
    # tag1 -> tag2
    #    P(t2|t1) = C(t1, t2) / C(t1)
    # example format on what to return:
    #    tag = 'NOUN', return {"NOUN": 0.300, "VERB": 0.450, "ADJ": 0.150}
    import itertools
    tagtag = list(itertools.product(utags, utags))

    tag_to_tag_probs_list = {}
    for tag1, tag2 in tagtag:
        count_tag1 = tag_unigrams[tag1]
        
        if tag1 not in tag_to_tag_probs_list:
            tag_to_tag_probs_list[tag1] = {}
        
        tag_to_tag_prob = tag_bigrams[(tag1, tag2)] / count_tag1
        tag_to_tag_probs_list[tag1][tag2] = tag_to_tag_prob
    
    return tag_to_tag_probs_list


# transition - basic_model.start -> tag 
none_to_start_tag_probabilities = tag_is_start_probs()
for start_tag, prob in none_to_start_tag_probabilities.items():
    # retrieve object created earlier in create state is word emission
    start_tag_state_obj = tag_state_list[start_tag]
    
    # add to model
    basic_model.add_transition(basic_model.start, start_tag_state_obj, prob)

# transition - tag -> basic_model.end
tag_to_end_probabilities = tag_is_end_probs()
for end_tag, prob in tag_to_end_probabilities.items():
    # retrieve object created earlier in create state is word emission
    end_tag_state_obj = tag_state_list[end_tag]
    
    # add to model
    basic_model.add_transition(end_tag_state_obj, basic_model.end, prob)

# transition - tag -> tag
# cross join the unique tags
# e.g. [tag1][tag1], [tag1][tag2], [tag1][tag3], [tag1][tag4], [tag1][tag5]
#      [tag2][tag1], [tag2][tag2], [tag2][tag3], [tag2][tag4], [tag2][tag5]
# import itertools
# tagtag = list(itertools.product(utags, utags))

tag_to_tag_probabilities = tag_to_tag_probs()
for tag1 in state_to_state_tag_probabilities.keys():
    for tag2 in tag_to_tag_probabilities[tag1].keys():
                
        prob = tag_to_tag_probabilities[tag1][tag2]

        tag1_state_obj = tag_state_list[tag1]
        tag2_state_obj = tag_state_list[tag2]

        basic_model.add_transition(tag1_state_obj, tag2_state_obj, prob)


####################################################################
# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [109]:
# test - explore basic_model

# basic_model.start
# {
#     "name" : "base-hmm-tagger-start",
#     "weight" : 1.0,
#     "distribution" : null,
#     "class" : "State"
# }

# basic_model.end
# {
#     "name" : "base-hmm-tagger-end",
#     "weight" : 1.0,
#     "distribution" : null,
#     "class" : "State"
# }

# tag_starts.keys()
# dict_keys(['ADP', 'X', 'ADJ', '.', 'PRT', 'CONJ', 'ADV', 'NOUN', 'NUM', 'DET', 'VERB', 'PRON'])

# dir(basic_model)

# basic_model.states
# data.tagset

import itertools
tagtag = list(itertools.product(utags, utags))
# print (tagtag)
# print (tagtag.count(('ADP', 'ADP')))    
# print (len(tagtag)) #144
print (len(set(tagtag))) #144


144


In [110]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
# code correction, i think code here met 'hmm_testing_acc' instead
# assert hmm_training_acc > 0.955, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."

HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

training accuracy basic hmm model: 97.49%
testing accuracy basic hmm model: 95.87%


### Example Decoding Sequences with the HMM Tagger

In [112]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Words:\n-----------------")
    print(data.sentences[key].words)
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Words:
-----------------
('and', 'August', '15', ',', 'November', '15', ',', 'February', '17', ',', 'and', 'May', '15', ',', '(', 'Cranston', ')', '.')
Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Words:
-----------------
('She', 'had', 'the', 'opportunity', 'that', 'few', 'clever', 'women', 'can', 'resist', ',', 'of', 'showing', 'her', 'superiority', 'in', 'argument', 'over', 'a', 'man', '.')
Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB


## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [31]:
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook HMM Tagger.ipynb to html',
 '[NbConvertApp] Writing 419128 bytes to HMM Tagger.html',
 '[NbConvertApp] Converting notebook HMM Tagger - Original.ipynb to html',
 '[NbConvertApp] Writing 327806 bytes to HMM Tagger - Original.html',
 '[NbConvertApp] Converting notebook HMM warmup (optional).ipynb to html',
 '[NbConvertApp] Writing 311506 bytes to HMM warmup (optional).html']

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [1]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]

[nltk_data] Downloading package brown to /home/fi/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]